<h1> Counting Summations </h1>

It is possible to write $5$ as a sum in exactly $6$ different ways:
\begin{align*}
    &4 + 1 \\
    &3 + 2 \\
    &3 + 1 + 1 \\
    &2 + 2 + 1 \\
    &2 + 1 + 1 + 1 \\
    &1 + 1 + 1 + 1 + 1 
\end{align*}

How many different ways can $100$ be written as a sum of at least $2$ positive integers?

---

The partition of integers can be described by the recurrence relation,
\begin{gather}
    p_n = \sum_{k=1}^{n} p_k(n) \\
    p_k(n) = p_k(n-k) + p_{k-1}(n-1) \\
    p(n) = \sum_{k \in \mathcal{z}} (-1)^{k+1} p(n-k(3k-1)/2)
\end{gather}
<!-- https://www.whitman.edu/mathematics/cgt_online/book/section03.03.html -->
<!-- https://en.wikipedia.org/wiki/Partition_function_(number_theory) -->

In [2]:
use std::ops::{Add, Mul, Div};

extern crate num;
use num::PrimInt;
use num::cast::ToPrimitive;

In [3]:
fn sum_first_n_natural_numbers<T> (n:T) -> T 
    where T: Add<Output = T> + Mul<Output = T> + Div<Output = T> + PrimInt
{
    let two = num::cast(2).unwrap();
    n*(n + T::one())/two
}

In [4]:
fn number_of_summations<T>(n: T) -> T 
    where T: Mul<Output = T> + Div<Output = T> + PrimInt + std::fmt::Debug
{
    if n == num::cast(2).unwrap() {
        return T::one()
    }
    
    let is_even : bool = n & num::one() == num::zero();
    let two: T = num::cast(2).unwrap();
    let half_n = n/two;
    
//     println!("{} {:?}", is_even, half_n);
    
    if is_even {
        two*sum_first_n_natural_numbers(half_n) - half_n
    } else { 
        two*sum_first_n_natural_numbers(half_n)  
    }
    
}

## Brute force

In [5]:
fn pure_recursive_partition_function(input_num: i32) -> i32 
{
    // Put lowest cases into match to reduce computational time
    match input_num {
        0 | 1 => return 1,
        2 => return 2,
        3 => return 3,
        4 => return 5,
        5 => return 7,
        6 => return 11,
        num if num < 0 => return 0,
        _ => ()
    };
    
    let mut sum = 0;
    let mut k = 1;
    
    loop {
        let x: i32 = (k*(3*k - 1))/2;
        
        if x > input_num + k {
            return sum;
        }
        
//         println!("input num = {} {} {}",input_num, input_num - x, input_num -x -k);
        
        let y = pure_recursive_partition_function(input_num - x) + pure_recursive_partition_function(input_num - x - k);
        
        if (k+1) & 1 == 1 {
            sum -= y
        } else {
            sum += y
        }
        
        k += 1;
    }
}

In [6]:
let soln = pure_recursive_partition_function(12);
println!("p(40) = {}", soln);

p(40) = 77


## A bit less brute force

In [7]:
fn higher_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num + num)/2) as u32
}
fn lower_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num - num)/2) as u32
}


In [8]:
fn maximum_k_val(input_num: u32) -> u32 {
    ((1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
}

In [9]:
fn compute_pentagonal_num(input_num: u32) -> Vec<u32> {
    let max_k = maximum_k_val(input_num);
    
    if max_k == 0 {
        return Vec::<u32>::new();
    }
    
    (1..=max_k).map(|k| higher_pentagonal_num(k))
                .into_iter()
                .chain(
                    (1..=max_k).map(|k| lower_pentagonal_num(k))
                    .into_iter())
                .collect()
}

In [10]:
use std::collections::HashSet;

fn compute_integers_from_pent_num(input_num: u32, pentagonal_nums: Vec<u32>) -> HashSet<u32> {
    
    let mut set = HashSet::with_capacity(input_num as usize);
    
    for pent_num in pentagonal_nums.iter() {
        let mut inner_counter = 1;
        
        loop {
            let n_time_k = inner_counter*pent_num;
            
            if n_time_k >= input_num {
                break
            } else {
                set.insert(input_num - n_time_k);
                inner_counter += 1;
            }
        }
    }
    
    set.shrink_to_fit();
    set
}

In [11]:
use std::collections::HashSet;

fn required_integers(input_num: u32) -> Vec<u32> {
    
    let pentagonal_nums = compute_pentagonal_num(input_num);
    println!("{:?}", pentagonal_nums);
    
    let set = compute_integers_from_pent_num(input_num, pentagonal_nums);
    
    let mut out = set.into_iter().collect::<Vec<_>>();
    out.sort();
    out
}

In [12]:
use std::collections::HashMap;
use std::convert::TryInto;

fn recursive_partition_function(input_num: u32, storage_map: &HashMap<u32, u32>) -> HashMap<u32, u32>
{
    let mut out = HashMap::with_capacity(1 + storage_map.len());
    
    if input_num < 7 {
        // Put lowest cases into match to reduce computational time
        let out_val = match input_num {0 | 1 => 1,
                                        2 => 2,
                                        3 => 3,
                                        4 => 5,
                                        5 => 7,
                                        6 => 11,
                                        _ => 0
        };
        out.insert(input_num, out_val);
        return out;
    }
    
    let max_k = maximum_k_val(input_num);
    let mut sum: i32 = 0;
    
    for k in 1..=max_k {
        let upper_val: i32 = input_num as i32 - lower_pentagonal_num(k) as i32;
        let lower_val: i32 = input_num as i32 - higher_pentagonal_num(k) as i32;
        
        let vals = [upper_val, lower_val];
//         println!("max k = {}, vals={:?}", max_k, vals);
        
        let mut partition_vals = vec![0; 2];
        
        for (counter, value) in vals.iter().enumerate() {
            
            if value == &0 {
                partition_vals[counter] = 1;
                continue
            } else if value < &0 {
                partition_vals[counter] = 0;
                continue
            }
            
            let val: u32 = *value as u32;
            
            let map_val = storage_map.get(&val);
            
            let num_partitions: u32;
            
            if map_val.is_none() && out.get(&val).is_none() {
                let sub_partition_val = pure_recursive_partition_function(*value as i32) as u32;
                num_partitions = sub_partition_val;
                out.insert(val, sub_partition_val);
            } else if out.get(&val).is_some() {
                num_partitions = *out.get(&val).unwrap();
            } else {
                num_partitions = *map_val.unwrap();
            }
            
            partition_vals[counter] = num_partitions;
        }
        
        
        let temp = (partition_vals[0] + partition_vals[1]) as i32;
        
//         println!("num={}, k={}, pval={:?}, psum={}", input_num, k, partition_vals, temp);
        
        if (k+1) & 1 == 1 {
            sum -= temp
        } else {
            sum += temp
        }
    }
    
    out.insert(input_num, sum as u32);
    
    out.extend(storage_map);
    
    out
}

In [13]:
fn not_in_storage(input_num: u32, storage_map: HashMap<u32, u32>) -> Vec<u32> {
    let pentagonal_nums = compute_pentagonal_num(input_num);
    pentagonal_nums.iter()
                    .map(|pent_num| input_num - pent_num)
                    .filter(|base_int| !storage_map.contains_key(base_int))
                    .collect::<Vec<u32>>()
}

In [14]:
fn build_base_partition_vals(partition_integers: Vec<u32>) -> HashMap<u32, u32> {
    
    let mut all_partition_vals: HashMap<u32, u32> = HashMap::new();
    
    for val in partition_integers.iter() {
        all_partition_vals = recursive_partition_function(*val, &all_partition_vals);
    }
    
    all_partition_vals
}

In [15]:
fn find_integer_partition(input_num: u32) -> u32 {
    
//     let recursive_integers = required_integers(input_num);
    let recursive_integers = (0..input_num).collect::<Vec<u32>>();
    
    let store_partition_vals: HashMap<u32, u32> = build_base_partition_vals(recursive_integers);
    
    let all_partition_vals = recursive_partition_function(input_num, &store_partition_vals);
    
    *all_partition_vals.get(&input_num).unwrap()
}

In [16]:
let soln = find_integer_partition(100);
println!("{:?}", soln - 1);

190569291
